In [1]:
import pandas as pd                       
import numpy as np                       
import tensorflow as tf                       
from keras.models import Sequential      

from keras.preprocessing import image            
from keras.preprocessing.image import ImageDataGenerator  

from tensorflow.keras.optimizers import Adam  
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout     
from keras.utils import np_utils  

In [2]:
emotion_data = pd.read_csv('../data_csv/fer2013/fer2013.csv')
print(emotion_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [3]:
X_train, y_train, X_test, y_test = [], [], [], []   

for index, row in emotion_data.iterrows():    
    k = row['pixels'].split(" ")       
    if row['Usage'] == 'Training':    
        X_train.append(np.array(k,'float32'))   
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k,'float32'))
        y_test.append(row['emotion'])

#print(X_train)
#print(y_train)
#print(type(X_test))
#print(type(y_test))

In [4]:
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

#print(type(X_test))
#print(type(y_test))

In [5]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)   
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)

#print(X_test)
#print(y_test)

In [6]:
#VGG11 Model
model = Sequential()

model.add(Conv2D(input_shape=(X_train.shape[1:]),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 256)       5

In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
emotion_model_info = model.fit(
    X_train,
    y_train,
    batch_size= 32,
    epochs= 30,
    verbose=1,
    validation_data=(X_test, y_test),
    shuffle=True)

Epoch 1/30
898/898 [==============================] - 377s 419ms/step - loss: 1.7103 - accuracy: 0.3086 - val_loss: 1.5065 - val_accuracy: 0.4191
Epoch 2/30
898/898 [==============================] - 376s 418ms/step - loss: 1.3960 - accuracy: 0.4585 - val_loss: 1.3074 - val_accuracy: 0.4993
Epoch 3/30
898/898 [==============================] - 376s 419ms/step - loss: 1.2216 - accuracy: 0.5322 - val_loss: 1.2518 - val_accuracy: 0.5224
Epoch 4/30
898/898 [==============================] - 377s 419ms/step - loss: 1.0711 - accuracy: 0.5912 - val_loss: 1.1835 - val_accuracy: 0.5561
Epoch 5/30
898/898 [==============================] - 377s 420ms/step - loss: 0.8969 - accuracy: 0.6686 - val_loss: 1.2015 - val_accuracy: 0.5665
Epoch 6/30
898/898 [==============================] - 384s 427ms/step - loss: 0.6924 - accuracy: 0.7491 - val_loss: 1.2609 - val_accuracy: 0.5854
Epoch 7/30
898/898 [==============================] - 417s 465ms/step - loss: 0.4848 - accuracy: 0.8284 - val_loss: 1.4056 -

In [9]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
  model.save_weights("model.h5")

print("Saved model to disk")

Saved model to disk
